In [3]:
from bs4 import BeautifulSoup
from requests import request, get, post

headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36' , 
           'Accept-Language' : 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'}
resp = get('https://www.coupang.com/np/categories/497135?page=1', headers = headers)

In [4]:
resp.status_code

200

In [5]:
dom = BeautifulSoup(resp.text, 'html5lib')

In [7]:
link = []

In [11]:
for elem in dom.select('a[class="baby-product-link"]'):
    link.append(elem.attrs['href'])

In [13]:
def prod_link(url, headers = None):
    links = []
    resp = get(url, headers = headers)
    dom = BeautifulSoup(resp.text, 'html5lib')
    for elem in dom.select('a[class="baby-product-link"]'):
        links.append(elem.attrs['href'])
    return links

In [15]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36' , 
           'Accept-Language' : 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'}

link_res = []
for page in range(1, 18):
    url = f'https://www.coupang.com/np/categories/497135?page={page}'
    link_res.extend(prod_link(url, headers))

In [57]:
link_res

['/vp/products/6645530847?itemId=15073113805&vendorItemId=82295358852&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/6372763434?itemId=19064151414&vendorItemId=86187002175&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/7217429026?itemId=18122375218&vendorItemId=85273457630&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/7371332906?itemId=19015540471&vendorItemId=86140097860&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/7167043809?itemId=18048173184&vendorItemId=85421826434&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/7352614847?itemId=18924345602&vendorItemId=86051363037&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/6662026640?itemId=15294714959&vendorItemId=82515116791&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/7389889714?itemId=19059391472&vendorItemId=86182358190&sourceType=CATEGORY&categoryId=497035',
 '/vp/products/7505816676?itemId=18629263312&vendorItemId=85764591502&sourceType=CATEGORY&categoryId=497035',
 '/vp/prod

In [19]:
import pickle

with open('laptop_coupang.pickle', 'wb') as f:
    pickle.dump(link_res, f)

In [36]:
resp = get('https://www.coupang.com/vp/products/6645530847?itemId=15073113805&vendorItemId=82295358852&sourceType=CATEGORY&categoryId=497035&isAddedCart=', headers=headers)
dom = BeautifulSoup(resp.text, 'html5lib')

In [102]:
import re
from urllib.request import urlretrieve
import os
import requests

def clean_filename(title):
    # 파일 이름에서 특수 문자와 공백을 제거하는 함수
    cleaned_title = re.sub(r'[\/:*?"<>|\s]', '', title)
    return cleaned_title

def img_url(url, headers):
    sess = requests.Session()
    resp = sess.get(url, headers=headers, verify=False)
    dom = BeautifulSoup(resp.text, 'html5lib')
    matches = list(set(re.findall(r'"origin":"(//[^"]+)"', str(dom))))
    
    return matches

def img_download(links, headers, fdir): # links = product links by 'prod_link' function
    if not os.path.exists(fdir):
        # 디렉토리가 존재하지 않으면 새로 생성
        os.makedirs(fdir)
        
    for link in links:
        link = 'https://www.coupang.com' + link
        img_link = img_url(link, headers = headers)
        for link_ in img_link:
            title = re.findall('/([^/]+)$', link_)[0]
            title = clean_filename(title)
            file_path = os.path.join(fdir, title)
            urlretrieve("https:" + link_, file_path)
        print('check link:', link)

In [103]:
img_download(link_res, headers, 'laptop')

C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.coupang.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


check link: https://www.coupang.com/vp/products/6645530847?itemId=15073113805&vendorItemId=82295358852&sourceType=CATEGORY&categoryId=497035


C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.coupang.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


URLError: <urlopen error [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1002)>